In [90]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.utils.data as data

torch.manual_seed(17)

# train_imgs_file = 'data/kmnist-npz/kmnist-train-imgs.npz'
# train_labels_file = 'data/kmnist-npz/kmnist-train-labels.npz'
batchsize = 50
epochs = 50
init_lr = 0.001

#   output[channel] = (input[channel] - mean[channel]) / std[channel]
#   output range(-1,1)<<-- input(0.5(mean),0.5(std))
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    # when mean=0.5,std=0.5 (inputmin(0)-0,5)/0.5=-1 & (inputmax(1)-0.5)/0.5=1
    # so output data range in (-1,1)    
    transforms.Normalize([0.5],[0.5]),
    # flip left and right    
    transforms.RandomHorizontalFlip(),
    # randomly crop image to 28 X 28 with 4 padding    
    transforms.RandomCrop(28,padding=4),
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5]),
])



In [46]:
import torch.utils.data as data
import pandas as pd
class MyDataset(data.Dataset):
    def __init__(self,imgs,labels, transform=None, target_transform=None):
        self.imgs = np.load(imgs)['arr_0']
        self.labels=np.load(labels)['arr_0']
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.imgs[idx,:,:]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    

In [47]:
training_set = MyDataset(
    imgs='data/kmnist-npz/kmnist-train-imgs.npz',
    labels='data/kmnist-npz/kmnist-train-labels.npz',
    transform=train_transforms,
    target_transform=None

)
testing_set = MyDataset(
    imgs='data/kmnist-npz/kmnist-test-imgs.npz',
    labels='data/kmnist-npz/kmnist-test-labels.npz',
    transform=test_transforms,
    target_transform=None,
)
valing_set = MyDataset(
    imgs='data/kmnist-npz/kmnist-val-imgs.npz',
    labels='data/kmnist-npz/kmnist-val-labels.npz',
    transform=val_transforms,
    target_transform=None
)

trainloader = data.DataLoader(training_set, batch_size=batchsize, shuffle=True)
testloader = data.DataLoader(testing_set, batch_size=batchsize, shuffle=True)
valloader = data.DataLoader(valing_set, batch_size=batchsize, shuffle=True)

In [92]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # convolutional layer(change channels' num)
        # H(output)=(H(input)−F+2P)/S+1
        # first conv:input shape (1,28,28)-> output shape (32,28,28)
        self.conv1 = nn.Conv2d(1,32,5,stride=1,padding=2)
        
        # first pooling:(32,28,28)->(32,14,14)
        
        # second conv: after pooling,input shape is (32,14,14) -> output shape (64,14,14)
        self.conv2 = nn.Conv2d(32,64,3,stride=1,padding=1)
        
        # second pooling:(64,14,14)->(64,7,7)
        
        # first linear connect: after second pooling,input shape is (64*7*7) -> output 1024
        self.fc1 = nn.Linear(7*7*64, 1024)
        # second linear connect:input 1024 -> output 10      
        self.fc2 = nn.Linear(1024 , 10)    
    
    def forward(self,x):   
    
    #  do first relu activation & pooling
        # first conv: output shape (batchsize,32,28,28)   
        res1 = F.relu(self.conv1(x))
        # pooling layer(change image size, usually 1/2)
        # H(output)=(H(input)−F)/S+1
        # first pooling: input shape (batchsize,32,28,28) -> output shape (batchsize,32,14,14)
        res2 = F.max_pool2d(res1,2)     
    
    #  second relu & pooling
        # second conv: output shape (batchsize,64,14,14)       
        res3 = F.relu(self.conv2(res2))
        # second pooling: input shape (batchsize,64,14,14) -> output shape (batchsize,64,7,7)
        res4 = F.max_pool2d(res3,2)
   
    #  full connectings & relu
        # (batchsize,64,7,7) -> output shape (batchsize,64*7*7) 
        res4 = res4.view(res4.shape[0], -1)
        # first full linear connection:(batchsize,64*7*7)->(batchsize,1024)       
        res5 = self.fc1(res4)
        # relu activation function        
        res6 = F.relu(res5)
        # second full linear connection:(batchsize,1024)->(batchsize,10)
        output = self.fc2(res6)      
        return output

In [93]:
def training(epochs,trainloader,valloader):
    train_losses=[]
    train_accs=[]
    val_accs=[]
    val_losses=[]
    CNN = Model()
    
    # use crossentroyLoss as loss function    
    loss_func = nn.CrossEntropyLoss()
    # set Adam optimizer with learning rate 0.001 and betas in (0.9,0.999)   
    optimizer = torch.optim.Adam(CNN.parameters(), lr=1e-3, betas=(0.9,0.999))
    
    running_loss=0.0
    train_acc=0.0
    val_acc=0.0
    lr=init_lr #0.001
    
    for epoch in range(epochs):
        running_loss=0.0
        accuracy=0.0
        train_acc=0.0
        total=0.0
        train_loss=0.0
        eachtotal=0.0
        
#         different learning rate with more epochs
        if epoch > 20:
            if epoch < 30:
                lr=0.5*init_lr #0.0005
            else:
                if epoch < 40:
                    lr=0.3*init_lr #0.0003
                
                elif epoch <45:
                    lr=0.1*init_lr #0.0001
                else:
                    lr=0.05*init_lr #0.00005
                    
        epochs = 50
        lr = 0.001
        for i ,data in enumerate(trainloader,0):
            inputs,labels = data
            
            optimizer.zero_grad()
            
            outputs = CNN(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss +=loss.item()
            train_loss += running_loss
            accuracy += (outputs.argmax(dim=1) == labels).sum().item()
            train_acc += accuracy
            eachtotal+=labels.size(0)
            total+=eachtotal
#             writer.add_scalar("acc vs epoch", trainAccSum/n, epoch)
            if i%100==100-1:
#                 writer.add_scalar("acc vs epoch", trainAccSum/n, epoch)
#                 print(f'running_loss:{running_loss}, acc:{trainAccSum/n}')
                print(running_loss)
                print(f'[{epoch +1},{i+1:5d}], acc:{100.*accuracy/eachtotal:.2f}% ,loss:{running_loss/eachtotal:.5f}')
                running_loss=0.0
                accuracy=0.0
                eachtotal=0.0
        
        PATH = './checkpoints/cifar_net_{:02d}.pth'.format(epoch)
        torch.save(CNN.state_dict(),PATH)
        
#         writer.add_scalar("acc vs epoch", train_acc/len(trainloader), epoch)
        train_accs.append(100*train_acc/total)
        print(f'training acc of epoch{epoch+1:2d}: {100.*train_acc/total:.2f}%')
        train_losses.append(train_loss/total)
        print(f'training loss of epoch{epoch+1:2d}: {train_loss/total:.5f}')
        
        validation_loss = 0.0
        running_loss = 0.0
        val_acc=0.0
        total=0.0
        print('starting validation for epoch {}'.format(epoch+1))
        with torch.no_grad():
            for data in valloader:
                inputs,labels = data            
                outputs = CNN(inputs)
                loss = loss_func(outputs, labels)
#                 validation_loss = loss.item() * len(data)
                validation_loss+=loss.item()
                val_acc += (outputs.argmax(dim=1) == labels).sum().item()
                total+=len(data[1])
        
            val_accs.append(100*val_acc/total)
            val_losses.append(validation_loss/total)
            print(f'validation loss for epoch{epoch+1:2d}: {validation_loss/total:.5f}')
            print(f'validation acc for epoch{epoch+1:2d}: {100.*val_acc/total:.2f}%')
            
    print('finished training')
    return train_accs,train_losses,val_accs,val_losses

In [86]:
# modify
def testing(testloader,epochs):
#     test_losses=[]
    test_accs=[]
    CNN = Model()
    for epoch in range(epochs):   
        CNN.load_state_dict(torch.load('./checkpoints/cifar_net_{:02d}.pth'.format(epoch)))
    
        correct=0.0
        total =0 
    
        with torch.no_grad():
            for data in testloader:
                images,labels = data
                outputs = CNN(images)
                correct += (outputs.argmax(dim=1) == labels).sum().item()
#             _,predicted=torch.max(outputs,data,1)
                total+=labels.size(0) 
#             correct+=(predicted==labels).sum().item()
        test_accs.append(100*correct/total)
        print(f'accuracy of the cnn epoch {epoch+1} on the testing images is :{100.*correct/total:.2f}%')
    
    return test_accs
            

In [87]:
def modify(valloader,epochs):
    CNN = Model()
    val_losses=[]
    val_accs=[]   
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        CNN.load_state_dict(torch.load('./checkpoints/cifar_net_{:02d}.pth'.format(epoch)))
        validation_loss = 0.0
        running_loss = 0.0
        val_acc=0.0
        total=0.0
        print('starting validation for epoch {}'.format(epoch+1))
        with torch.no_grad():
            for data in valloader:
                inputs,labels = data            
                outputs = CNN(inputs)
                loss = loss_func(outputs, labels)
                validation_loss+=loss.item()
                val_acc += (outputs.argmax(dim=1) == labels).sum().item()
                total+=len(data[1])
        
        val_accs.append(100*val_acc/total)
        val_losses.append(validation_loss/total)
        print(f'validation loss for epoch{epoch+1:2d}: {validation_loss/total:.5f}')
        print(f'validation acc for epoch{epoch+1:2d}: {100.*val_acc/total:.2f}%')
    return val_accs,val_losses

In [191]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer2 = SummaryWriter()

def display1(train_accs,train_losses,val_accs,val_losses):
    for i,train_acc in enumerate(train_accs,0):
        
        writer.add_scalar("train_acc vs epoch", train_acc, i+1)
        writer.add_scalar("train_loss vs epoch", train_losses[i], i+1)    
        writer.add_scalar("validation_acc vs epoch", val_accs[i], i+1)
        writer.add_scalar("validation_loss vs epoch", val_losses[i], i+1)
        
def display2(train1_accs):
    for i,train_acc in enumerate(train1_accs,0):
        writer.add_scalar("accuracies of different lr(0.001vs0.0001)", train_acc, i+1)
        
def display3(train2_accs):
    for i,train_acc in enumerate(train2_accs,0):
        writer.add_scalar("accuracies of different lr(0.001vs0.0001)", train_acc, i+1)
        
# def displayModify(val_accs,val_losses):
#     for i,val_acc in enumerate(val_accs,0):  
#         writer.add_scalar("validation_acc vs epoch", val_acc, i+1)
#         writer.add_scalar("validation_loss vs epoch", val_losses[i], i+1)

def display4(train1_accs):
    for i,train1_acc in enumerate(train1_accs,0):
        writer2.add_scalar("lr=0.001~0.00005_vs_lr=0.0005", train1_acc, i+1)

def display5(train2_accs):
    for i,train2_acc in enumerate(train2_accs,0):
        writer2.add_scalar("lr=0.001~0.00005_vs_lr=0.0005", train2_acc, i+1)        
        

In [89]:
train_accs,train_losses,val_accs,val_losses=training(epochs,trainloader,valloader)

161.85512244701385
[1,  100], acc:43.72% ,loss:0.03237
104.63437217473984
[1,  200], acc:64.78% ,loss:0.02093
79.07829701900482
[1,  300], acc:74.14% ,loss:0.01582
62.02160423994064
[1,  400], acc:80.28% ,loss:0.01240
51.89958652853966
[1,  500], acc:83.50% ,loss:0.01038
44.6718730032444
[1,  600], acc:85.80% ,loss:0.00893
40.46551941335201
[1,  700], acc:87.56% ,loss:0.00809
36.55056618154049
[1,  800], acc:88.38% ,loss:0.00731
33.87684143334627
[1,  900], acc:89.44% ,loss:0.00678
32.86215029656887
[1, 1000], acc:89.44% ,loss:0.00657
29.225905887782574
[1, 1100], acc:90.64% ,loss:0.00585
training acc of epoch 1: 79.33%
training loss of epoch 1: 0.01257
starting validation for epoch 1
validation loss for epoch 1: 0.00456
validation acc for epoch 1: 93.40%
27.441892445087433
[2,  100], acc:91.14% ,loss:0.00549
24.533755905926228
[2,  200], acc:91.74% ,loss:0.00491
25.17391838133335
[2,  300], acc:91.88% ,loss:0.00503
22.55440608598292
[2,  400], acc:93.02% ,loss:0.00451
25.2869950085878

5.988879087148234
[11, 1100], acc:98.12% ,loss:0.00120
training acc of epoch11: 97.79%
training loss of epoch11: 0.00145
starting validation for epoch 11
validation loss for epoch11: 0.00188
validation acc for epoch11: 98.20%
6.648374279611744
[12,  100], acc:97.96% ,loss:0.00133
6.670778114581481
[12,  200], acc:97.90% ,loss:0.00133
7.917735551483929
[12,  300], acc:97.74% ,loss:0.00158
6.513622018508613
[12,  400], acc:97.96% ,loss:0.00130
7.511511585209519
[12,  500], acc:97.68% ,loss:0.00150
6.757606732193381
[12,  600], acc:97.86% ,loss:0.00135
7.991994526819326
[12,  700], acc:97.50% ,loss:0.00160
6.560555697535165
[12,  800], acc:97.90% ,loss:0.00131
6.795860648635426
[12,  900], acc:97.84% ,loss:0.00136
5.9732737322337925
[12, 1000], acc:98.02% ,loss:0.00119
6.402266706689261
[12, 1100], acc:97.88% ,loss:0.00128
training acc of epoch12: 97.82%
training loss of epoch12: 0.00141
starting validation for epoch 12
validation loss for epoch12: 0.00133
validation acc for epoch12: 98.2

4.2258891272649635
[22,  500], acc:98.70% ,loss:0.00085
4.768454362230841
[22,  600], acc:98.50% ,loss:0.00095
4.958831179246772
[22,  700], acc:98.46% ,loss:0.00099
3.5431033945351373
[22,  800], acc:98.60% ,loss:0.00071
3.6793124213872943
[22,  900], acc:98.86% ,loss:0.00074
4.626408583309967
[22, 1000], acc:98.46% ,loss:0.00093
5.878277501324192
[22, 1100], acc:98.40% ,loss:0.00118
training acc of epoch22: 98.52%
training loss of epoch22: 0.00093
starting validation for epoch 22
validation loss for epoch22: 0.00172
validation acc for epoch22: 98.20%
3.869122405187227
[23,  100], acc:98.72% ,loss:0.00077
4.7362569338292815
[23,  200], acc:98.28% ,loss:0.00095
4.217083863506559
[23,  300], acc:98.66% ,loss:0.00084
3.9166463884175755
[23,  400], acc:98.70% ,loss:0.00078
4.73247035042732
[23,  500], acc:98.46% ,loss:0.00095
5.205982967003365
[23,  600], acc:98.32% ,loss:0.00104
4.188723296392709
[23,  700], acc:98.74% ,loss:0.00084
5.12283762881998
[23,  800], acc:98.40% ,loss:0.00102
5

validation loss for epoch32: 0.00200
validation acc for epoch32: 97.80%
2.992056869523367
[33,  100], acc:99.08% ,loss:0.00060
2.8360567753843497
[33,  200], acc:99.14% ,loss:0.00057
3.9862767858012376
[33,  300], acc:99.02% ,loss:0.00080
3.582160227393615
[33,  400], acc:98.82% ,loss:0.00072
4.168001689351513
[33,  500], acc:98.88% ,loss:0.00083
3.0576690562993463
[33,  600], acc:99.04% ,loss:0.00061
3.0708623314276338
[33,  700], acc:99.04% ,loss:0.00061
4.063840450748103
[33,  800], acc:98.80% ,loss:0.00081
3.8427403172900085
[33,  900], acc:98.86% ,loss:0.00077
3.2126572280249093
[33, 1000], acc:99.00% ,loss:0.00064
3.937004292230995
[33, 1100], acc:98.68% ,loss:0.00079
training acc of epoch33: 98.97%
training loss of epoch33: 0.00066
starting validation for epoch 33
validation loss for epoch33: 0.00180
validation acc for epoch33: 98.20%
2.758793169661658
[34,  100], acc:98.92% ,loss:0.00055
3.529487866184354
[34,  200], acc:98.72% ,loss:0.00071
3.427687304723804
[34,  300], acc:98

3.120859943661344
[43,  700], acc:99.04% ,loss:0.00062
3.446834259229945
[43,  800], acc:99.04% ,loss:0.00069
3.4755818769481266
[43,  900], acc:98.94% ,loss:0.00070
3.7880454066980747
[43, 1000], acc:98.80% ,loss:0.00076
2.9692062515896396
[43, 1100], acc:99.08% ,loss:0.00059
training acc of epoch43: 98.99%
training loss of epoch43: 0.00069
starting validation for epoch 43
validation loss for epoch43: 0.00203
validation acc for epoch43: 98.00%
3.266724635974242
[44,  100], acc:98.96% ,loss:0.00065
3.418499433551915
[44,  200], acc:98.92% ,loss:0.00068
3.1912346048702602
[44,  300], acc:99.18% ,loss:0.00064
2.289742736342305
[44,  400], acc:99.30% ,loss:0.00046
3.177657239866676
[44,  500], acc:99.06% ,loss:0.00064
3.221674602587882
[44,  600], acc:98.98% ,loss:0.00064
2.7615798578635804
[44,  700], acc:98.92% ,loss:0.00055
3.297773599078937
[44,  800], acc:98.90% ,loss:0.00066
3.7767906561202835
[44,  900], acc:98.80% ,loss:0.00076
3.999650288871635
[44, 1000], acc:98.72% ,loss:0.0008

In [94]:
test_accs=testing(testloader,epochs)

accuracy of the cnn epoch 1 on the testing images is :83.75%
accuracy of the cnn epoch 2 on the testing images is :88.83%
accuracy of the cnn epoch 3 on the testing images is :91.10%
accuracy of the cnn epoch 4 on the testing images is :92.97%
accuracy of the cnn epoch 5 on the testing images is :92.83%
accuracy of the cnn epoch 6 on the testing images is :93.26%
accuracy of the cnn epoch 7 on the testing images is :94.56%
accuracy of the cnn epoch 8 on the testing images is :94.11%
accuracy of the cnn epoch 9 on the testing images is :94.05%
accuracy of the cnn epoch 10 on the testing images is :95.05%
accuracy of the cnn epoch 11 on the testing images is :95.07%
accuracy of the cnn epoch 12 on the testing images is :95.45%
accuracy of the cnn epoch 13 on the testing images is :95.16%
accuracy of the cnn epoch 14 on the testing images is :95.56%
accuracy of the cnn epoch 15 on the testing images is :95.10%
accuracy of the cnn epoch 16 on the testing images is :95.82%
accuracy of the c

In [53]:
display(train_accs,train_losses,val_accs,val_losses)

In [193]:
test=np.loadtxt('test.txt')
test=test*50
test=test[20:]
train=list(train_accs[:])
train=[train[i]*50 for i in range(len(train))]
train=train[20:]
display4(test)
display5(train)
# display2(train_accs)
# display3(train2_accs)

In [197]:
# writer3 = SummaryWriter()
# for i,test_acc in enumerate(test_accs,0):
#         writer3.add_scalar("lr=0.001~0.00005_vs_lr=0.0005", test_acc, i+1)
        



4956.086051751604


In [80]:
# Mval_accs,Mval_losses=modify(valloader,epochs)

starting validation for epoch 1
validation loss for epoch 1: 0.00456
validation acc for epoch 1: 93.40%
starting validation for epoch 2
validation loss for epoch 2: 0.00314
validation acc for epoch 2: 95.60%
starting validation for epoch 3
validation loss for epoch 3: 0.00257
validation acc for epoch 3: 96.70%
starting validation for epoch 4
validation loss for epoch 4: 0.00225
validation acc for epoch 4: 96.70%
starting validation for epoch 5
validation loss for epoch 5: 0.00197
validation acc for epoch 5: 97.10%
starting validation for epoch 6
validation loss for epoch 6: 0.00206
validation acc for epoch 6: 96.60%
starting validation for epoch 7
validation loss for epoch 7: 0.00194
validation acc for epoch 7: 97.50%
starting validation for epoch 8
validation loss for epoch 8: 0.00249
validation acc for epoch 8: 97.50%
starting validation for epoch 9
validation loss for epoch 9: 0.00197
validation acc for epoch 9: 97.70%
starting validation for epoch 10
validation loss for epoch10: 0.

In [82]:
displayModify(Mval_accs,Mval_losses)